# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
The role of affect in the Automatic Approach Bias: An empirical investigation	Talk	talk-1	Philosophy of Science Conference -  Cognitive Science track, Inter-University Centre Dubrovnik (IUC)	2022-04-26	Dubrovnik, Croatia		
Computational approaches to a cognitive conception of language:	Talk	talk-2	Computational Cognitive Science Group at Donders Centre for Cognition/Radboud University	2022-05-09	Nijmegen, Netherlands (online)		
The role of language and pragmatics in higher-level cognition: Forming abstract concepts in social interaction	Short talk	talk-3	International Interdisciplinary Computational Cognitive Science Summer School (IICCSSS)	2022-09-07	Tübingen, Germany		
Feeling Good, Approaching the Positive: The Role of Affect and Arousal in the Approach-Avoidance Bias	Talk	talk-4	Doctorate Colloquium, Osnabrück University	2023-02-07	Osnabrück, Germany		
Replications in Experimental Linguistics	Invited talk	talk-5	The Frida

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,The role of affect in the Automatic Approach B...,Talk,talk-1,Philosophy of Science Conference - Cognitive ...,2022-04-26,"Dubrovnik, Croatia",NaN,NaN
1,Computational approaches to a cognitive concep...,Talk,talk-2,Computational Cognitive Science Group at Donde...,2022-05-09,"Nijmegen, Netherlands (online)",NaN,NaN
2,The role of language and pragmatics in higher-...,Short talk,talk-3,International Interdisciplinary Computational ...,2022-09-07,"Tübingen, Germany",NaN,NaN
3,"Feeling Good, Approaching the Positive: The Ro...",Talk,talk-4,"Doctorate Colloquium, Osnabrück University",2023-02-07,"Osnabrück, Germany",NaN,NaN
4,Replications in Experimental Linguistics,Invited talk,talk-5,"The Friday Primes, Humboldt University Berlin",2023-05-26,"Berlin, Germany",NaN,NaN
5,Pragmatics in referential communication: An in...,Contributed talk,talk-6,Procedural and computational models of semanti...,2023-08-02,"Ljubljana, Slowenia",NaN,NaN
6,Pragmatics in the referential communication of...,Poster,poster-1,The 10th biennial meeting of Experimental Prag...,2023-09-20,"Paris, France",NaN,NaN
7,Investigating concept communication in a multi...,Talk,talk-7,Workshop on the role of loopholes in communica...,2023-09-29,"Osnabrück, Germany",NaN,NaN
8,Pragmatics in the referential communication of...,Poster,poster-2,Computational Cognition (ComCo) Conference,2023-10-09,"Osnabrück, Germany",NaN,NaN
9,Two approaches to studying abstraction via con...,Talk,talk-8,ABSTRACTION group meeting,2023-11-20,"Bologna, Italy (online)",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2022-04-26-talk-1.md   2023-09-29-talk-7.md   2024-07-25-talk-12.md
2022-05-09-talk-2.md   2023-10-09-poster-2.md 2024-08-07-talk-13.md
2022-09-07-talk-3.md   2023-11-20-talk-8.md   2024-08-13-talk-14.md
2023-02-07-talk-4.md   2023-12-05-talk-9.md   2024-09-05-talk-15.md
2023-05-26-talk-5.md   2024-04-19-talk-10.md  2024-09-27-poster-4.md
2023-08-02-talk-6.md   2024-05-14-talk-11.md  2024-10-01-talk-16.md
2023-09-20-poster-1.md 2024-05-22-poster-3.md


In [18]:
!cat ../_talks/2023-02-07-talk-4.md

---
title: "Feeling Good, Approaching the Positive: The Role of Affect and Arousal in the Approach-Avoidance Bias"
collection: talks
type: "Talk"
permalink: /talks/2023-02-07-talk-4
venue: "Doctorate Colloquium, Osnabrück University"
date: 2023-02-07
location: "Osnabrück, Germany"
---
